# OpenFace Sonification with Panson

This notebook will introduce **panson**: a framework for interactive sonification based on **sc3nb**.

Connecting panson with OpenFace, we will implement several examples of sonification of facial expressions, head rotations and gaze, while showing some of the potentialities of panson.

In [ ]:
import numpy as np
import pandas as pd

import subprocess
import time
import os

import panson as ps
from panson import bundle

from math import pi, exp, log2
from collections import namedtuple

## Setup

### OpenFace docker container

The following section assumes that OpenFace was installed using **docker**.

**Run the container** with the following command:

`docker run -it --rm --name openface --mount type=bind,source=/mount/dir,target=/home/openface-build/files algebr/openface:latest`

Substitute /mount/dir with the absolute path of the mount point. This directory is where you need to put the video files to make sure that the OpenFace executable can access them. This is because the directory is shared between the file system of the host and the one of the container.

Now we can launch OpenFace executables (present in the container) from outside the container with a command similar to the following:

`docker exec -it openface build/bin/FeatureExtraction -out_dir files/processed -f files/video.avi`

The output directory files/processed must be created in advance.

Later in the notebook, python functions are provided to run the executables of interest.

When called in live mode (e.g. while perform feature extraction from a webcam), the executables will attempt to display a window, and fail with an error if not possible.

If your sistem is running the X server as windowing system, the following cell will allow all programs to make connections with the X server. This will allow the container to display windows, but it is a workaround. It is not safe in the general case.

In [ ]:
!xhost +

The following functions are used to interact with OpenFace executables.

**feature_extraction_online** requires some explanation. OpenFace does not provide any supported way of streaming live features: when the executable is started in real-time mode, all it will do is write the features to an output .csv file. In the original plan of development of the project, a feature streaming server should have been implemented, making the executables capable of streaming features through the network. This was never implemented though.

The workaround found is to make the executable write to a named pipe rather than to an ordinary file. Panson will be instructed to read data from the named pipe.

In [ ]:
# name given to the container
CONTAINER_NAME = 'openface'

# base directory of the container
CONTAINER_BASE_DIR = '/home/openface-build'
# directory with executalbles in the container
CONTAINER_BIN_DIR = os.path.join(CONTAINER_BASE_DIR, 'build/bin')

FILE_DIR = '../media/files'
OUT_DIR = os.path.join(FILE_DIR, 'processed')

CONTAINER_FILE_DIR = os.path.join(CONTAINER_BASE_DIR, 'files')
CONTAINER_OUT_DIR = os.path.join(CONTAINER_FILE_DIR, 'processed')

CONTAINER_EXECUTABLE = os.path.join(CONTAINER_BIN_DIR, 'FeatureExtraction')


def feature_extraction_offline(video_name):
    """Perform feature extraction on video file."""
    
    video_path = os.path.join(FILE_DIR, video_name)
    
    # the file must be in FILE_DIR
    if not os.path.isfile(video_path):
        raise FileNotFoundError(video_path)
    
    container_video_path = os.path.join(CONTAINER_FILE_DIR, video_name)
    
    command = [
        'docker', 'exec', CONTAINER_NAME, CONTAINER_EXECUTABLE,
        '-f', container_video_path,
        '-out_dir', CONTAINER_OUT_DIR,
        # features extracted
        '-pose', '-gaze', '-aus',
        # output tracked video
        '-tracked'
    ]
    
    # capture and combine stdout and stderr into one stream and set as text stream
    proc = subprocess.Popen(command,stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
        
    # poll process and show its output
    while True:
        output = proc.stdout.readline()
        
        if output:
            print(output.strip())
            
        if proc.poll() is not None:
            break
    
    return proc

def feature_extraction_online(pipe='files/pipe'):
    """Start online feature extraction and return."""
    
    command = [
        'docker', 'exec', CONTAINER_NAME, CONTAINER_EXECUTABLE,
        '-device', '0', # use default device
        '-pose', '-gaze', '-aus',
        # '-tracked'
        '-of', pipe
    ]
    
    # capture and combine stdout and stderr into one stream and set as text stream
    proc = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)    

    print('Starting real-time analysis...')
    print('Open the pipe from the read side to start the feature stream')
    
    return proc

def kill_feature_extraction_online():
    """Kill online feature extraction."""
    
    # !docker exec -it openface pkill FeatureExt
    command = ['docker', 'exec', CONTAINER_NAME, 'pkill', 'FeatureExt']
    subprocess.run(command)

def read_openface_csv(csv_path):
    """Read csv files produced by openface executables."""
    
    return pd.read_csv(csv_path, sep=r',\s*', engine='python')

When executing **offline feature extraction**, sometimes OpenFace will drop some frames without processing them, causing the output data (and tracked video output) to be inaccurate. This may be related to OpenCV not being able to work with some of the codecs, but in general this is an open issue of OpenFace.

One workaround is to:
1. Split video frames into separate images
    * `ffmpeg -i video.avi video_dir/frame%04d.jpg`
2. Instruct OpenFace to process the frame directory with the **-fdir** option
    * The csv output of openface will not contain timestamp information, so we would have to recreate them from frame files names
    
This notebook will mainly focus on real-time processing, as that's the most interesting part as concerns the framework. For this reason we will not use this workaround for offline processing.

The following functions are used to perform operations on video files using **ffmpeg**.

In [ ]:
def ffmpeg_convert(in_file, out_file):
    """Convert video file from one format to the other."""

    command = [
        'ffmpeg', '-y',
        '-i', in_file,
        '-c:v', 'libx264',
        '-crf', '22',
        '-pix_fmt', 'yuv420p',
        '-c:a', 'libvo_aacenc',
        '-b:a', '128k',
        out_file
    ]
    
    proc = subprocess.run(command, capture_output=True)
    
    return proc

def ffmpeg_merge(video_file, audio_file, out_file):
    """Merge audio and video files."""
    
    # ffmpeg -i files/phone-processed.mp4 -i score.wav  -c:v copy phone-processed-son.mp4 -y
                
    command = [
        'ffmpeg', '-y',
        '-i', video_file,
        '-i', audio_file,
        '-map', '0:v',
        '-map', '1:a',
        '-c:v', 'copy',
        out_file
    ]
    
    proc = subprocess.run(command, capture_output=True)
    
    return proc

### Supercollider / sc3nb

In [ ]:
import sc3nb as scn

In [ ]:
# start scsynth
sc = scn.startup()

# connect scsynth to the system playback
!jack_connect "SuperCollider:out_1" "system:playback_1"
!jack_connect "SuperCollider:out_2" "system:playback_2"

If **jack_connect** is not available on your sistem, you can use other programmes to connect SuperCollider to the system output. For instance, with **QJackCtl** you can graphically link the nodes to do this.

In [ ]:
# sc.exit()

Test SuperCollider output.

In [ ]:
sc.server.blip()

Set latency of the server. This step is mandatory; the value should be appropriate with respect to the user's configuration and messages sent by the framework should be received on time.

In [ ]:
sc.server.latency = 0.1

## Panson

### Offline

In this way it is possible to load the feature extraction data produced by OpenFace executables.

In [ ]:
# the path is relative to the docker container
csv_file = "/path/to/file.csv"

In [ ]:
df = read_openface_csv(os.path.join(OUT_DIR, csv_file))
df.head()

### Online

When working with real-time data we will need the object declared hereafter. It is a **stream** object that allow the panson to get data from OpenFace executables (in real-time).

FIFO_PATH is the path of the named pipe where OpenFace is writing its live feature extraction data. **feature_extraction_online** (added as an opening hook) writes by default to **files/pipe.csv**.

The test method tries to get some data from the stream and returns some information about it.

In [ ]:
from panson import streams

FIFO_PATH = os.path.join(FILE_DIR, 'pipe.csv')

openface_stream = streams.CsvFifo('openface', args=(FIFO_PATH,))\
        .add_open_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)\
        .test()

In [ ]:
openface_stream.sample_size, openface_stream.dtype, openface_stream.fps

### Tutorials

#### Sonification

Hereafter, there are some implementations in different coding styles of a continuous dummy sonification of AU 4. The intensity of the AU is simply mapped to the frequency of a continuous synth (we use the default synth s2).

In [ ]:
S2_PATH = os.path.join(scn.resources.__path__[0], "synthdefs", "s2.scsyndef")

It is possible to access the server instance of the sonification through the attribute **s**. This will be the default server most of the time.

As the we don't want the sonification to have parameters, we will neglect the **init_parameters** method. Look later on in this notebook for examples of its usage.

In **init_server** we load the server resources, such as synthdefs and buffers. Even if default synthdef may be loaded by default at server startup, we need to load it anyway explicitly to make NRT sonification (export) work properly.

#### Message-style (explicit ID allocation)

##### Explicit bundling

In [ ]:
from sc3nb.osc.osc_communication import Bundler

# message style (explicit ID allocation)
class AU04ContinuousSonification(ps.Sonification):
    
    def init_parameters(self):
        pass
    
    def init_server(self):
        bundler = Bundler()
        bundler.add(0, "/d_load", [S2_PATH])
        return bundler
    
    def start(self):
        bundler = Bundler()
        self.au4_node_id = self.s.node_ids.allocate(1)[0]        
        bundler.add(0, "/s_new", ["s2", self.au4_node_id, 0, 0, "amp", 0])
        return bundler
    
    def stop(self, server):
        bundler = Bundler()
        self.s.node_ids.free([self.au4_node_id])    # actually this does nothing
        bundler.add(0, "/g_freeAll", [0])
        return bundler
    
    def _process(self, row):
        bundler = Bundler()
        # only "max" should be enough (to clip the top part to 0.3)
        amp = scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax")
        # map the intensity of the AU in one octave range
        freq = scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
        bundler.add(0, "/n_set", [self.au4_node_id, "amp", amp, "freq", freq])
        return bundler

##### Implicit bundling

The **bundle decorator** can be used to capture the messages produced by the methods and return them in a bundler object.

In [ ]:
class AU04ContinuousSonification(ps.Sonification):
    
    def init_parameters(self):
        pass
    
    @bundle
    def init_server(self):
        self.s.msg("/d_load", [S2_PATH], bundle=True)
    
    @bundle
    def start(self):
        self.au4_node_id = self.s.node_ids.allocate(1)[0]
        self.s.msg("/s_new", ["s2", self.au4_node_id, 0, 0, "amp", 0], bundle=True)
    
    @bundle
    def stop(self):
        self.s.node_ids.free([self.au4_node_id])    # actually this does nothing
        self.s.msg("/g_freeAll", [0], bundle=True)
    
    @bundle
    def _process(self, row):
        # only "max" should be enough (to clip the top part to 0.3)
        amp = scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax")
        # map the intensity of the AU in one octave range
        freq = scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
        self.s.msg("/n_set", [self.au4_node_id, "amp", amp, "freq", freq], bundle=True)

#### High-level style (implicit ID allocation)

##### Explicit bundling

In [ ]:
from sc3nb.osc.osc_communication import Bundler
from sc3nb import SynthDef


class AU04ContinuousSonification(ps.Sonification):
    
    def init_parameters(self):
        pass

    def init_server(self):
        with Bundler(send_on_exit=False) as bundler:
            SynthDef.load(S2_PATH)
        return bundler

    def start(self):
        with Bundler(send_on_exit=False) as bundler:
            self.synth = scn.Synth("s2", {"amp": 0})
        return bundler
    
    def stop(self):
        with Bundler(send_on_exit=False) as bundler:
            self.s.free_all()
        return bundler
    
    def _process(self, row):
        with Bundler(send_on_exit=False) as bundler:
            self.synth.set(
                # only "max" should be enough (to clip the top part to 0.3)
                "amp", scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax"),
                # map the intensity of the AU in one octave range
                "freq", scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
            )
        return bundler

##### Implicit bundling

This is the most high level programming style that sc3nb allows, hence it is the **recommended**.

In [ ]:
from sc3nb import SynthDef

# a (implicit ID allocation)
class AU04ContinuousSonification(ps.Sonification):
        
    def init_parameters(self):
        pass

    @bundle
    def init_server(self):
        SynthDef.load(S2_PATH)

    @bundle
    def start(self):
        self.synth = scn.Synth("s2", {"amp": 0})
    
    @bundle
    def stop(self):
        self.s.free_all()
    
    @bundle
    def _process(self, row):
        self.synth.set(
            # only "max" should be enough (to clip the top part to 0.3)
            "amp", self.amp * scn.linlin(row[self.label], 0, 1, 0, 0.3, "minmax"),
            # map the intensity of the AU in one octave range
            "freq", scn.midicps(scn.linlin(row[self.label], 0, 5, 69, 81))
        )

#### Preprocessing

It is possible to specify live preprocessing algorithms to be used in real-time by subclassing the **Preprocessor** abstract class.

The following example will define a moving average with a window size of 10. This preprocessor will change the values of every data row obtained from the stream.

In [ ]:
# features we want to compute the average of (AU intensities, head pose and gaze)
avg_features = df.filter(regex='(AU.{2}_r)|(pose_R.)|(gaze_angle_.)').columns.to_list()
# avg_features

In [ ]:
class MovingAverage(ps.Preprocessor):
    
    def __init__(self):
        self.avg_features = avg_features
        
        self.window_size = 10
        self.window = pd.DataFrame()
    
    def preprocess(self, row: pd.Series):            
        self.window = self.window.append(row[self.avg_features])
    
        if self.window.shape[0] > self.window_size:
            self.window = self.window.drop(self.window.index[0])

        mean_series = self.window.mean()

        # write values into row
        for label in self.avg_features:
            row[label] = mean_series[label]

In [ ]:
avg_openface_stream = streams.CsvFifo('openface', args=(FIFO_PATH,), preprocessor=MovingAverage)\
        .add_open_hook(feature_extraction_online)\
        .add_close_hook(kill_feature_extraction_online)\
        .test()

## Sonifications

### Sonification: AU04 Test

This sonification is similar to the previous sonifications, but with the addition of a parameter for amplitude regulation.

This is a very simple sonification of AU04 (Brow Lowerer). The intensity of AU04 is used here to modulate both the amplitude and the frequency of a continuous synth. As continuous synth, the default synth of sc3nb s2 is used (we will have to instruct the server to load it).

* The intensity range \[0,1\] is mapped into the amplitude range \[0,0.3\], where 0.3 will be the maximum amplitude of the sound. The sonification has a parameter amp that can be used to scale this range.
* The intensity range \[0,5\] is mapped into the midi range \[69,81\]

In [ ]:
class AU04ContinuousSonification(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
        
    def init_parameters(self):
        self.amp = 1
    
    @bundle
    def init_server(self):
        self.s.load_synthdefs()

    @bundle
    def start(self):
        # lag time is decided based on the frame rate
        self.synth = scn.Synth("s2", {"amp": 0, "lg": 0.03})

    @bundle
    def _process(self, row):  
        self.synth.set(
            # only "max" should be enough (to clip the top part to 0.3)
            "amp", self.amp * scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax"),
            # map the intensity of the AU in one octave range
            "freq", scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
        )

In [ ]:
son = AU04ContinuousSonification()
son

#### Offline

We can play the sonification on offline data along with the original video.

In [ ]:
vp = ps.VideoPlayer('path/to/video', fps=30)

In [ ]:
feature_display = ps.RTFeatureDisplay(['AU04_r', 'AU12_r'], queue_size=50)
# dp = ps.DataPlayer(son, feature_display=feature_display, video_player=vp).load(df)
dp = ps.DataPlayer(son, feature_display=feature_display).load(df)

In [ ]:
# feature_display.show(fps=30)
# display(son)
display(dp)

In [ ]:
vp.quit()

#### Online

Play sonification on real-time data using **openface_stream** (defined before).

In [ ]:
son = AU04ContinuousSonification()
son

In [ ]:
feature_display = ps.RTFeatureDisplay(['AU04_r'], 80)
rtdp = ps.RTDataPlayer(openface_stream, son, feature_display=feature_display)

In [ ]:
feature_display.show(fps=10)
display(son)
rtdp

### Sonification: Pentatonic eyes + brows

Here we are considering only the upper part of the face. More precisely, the following AUs.
* 1: Inner Brow Raiser
* 2: Outer Brow Raiser (unilateral)
* 4: Brow Lowerer
* 5: Upper Lid Raiser
* 6: Cheek Raiser
* 7: Lid Tightener

With the following sonification, we'll be able to control some synths playing a minor pentatonic scale with the upper part of our faces.

In [ ]:
class PentatonicContinuous(ps.Sonification):

    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
    fundamental = ps.MidiSliderParameter()
    
    AUs_offsets = {
        'AU01_r': 12,
        'AU02_r': 10,
        'AU04_r': 7,
        'AU05_r': 5,
        'AU06_r': 0,
        'AU07_r': 3
    }

    def init_parameters(self):
        self.amp = 0.3
        self.fundamental = 69

    @bundle
    def init_server(self):
        self.s.load_synthdefs()

    @bundle
    def start(self):
        self.synths = {}
        
        for label, offset in self.AUs_offsets.items():
            self.synths[label] = scn.Synth("s2", {"amp": 0, "freq": scn.midicps(self.fundamental + offset), "lg": 0.015})
    
    @bundle
    def _process(self, row):
        for label, offset in self.AUs_offsets.items():
            intensity = row[label]
            amp = self.map_intensity(intensity)
            self.synths[label].set(
                "amp", self.amp * amp,
                "freq", scn.midicps(self.fundamental + offset)
            )
            
        
    @staticmethod
    def map_intensity(intensity):
        if intensity < 1.0:
            amp = 0
        else:
            db = scn.linlin(intensity, 1, 5, -20, -5, "minmax")
            amp = scn.dbamp(db)

        return amp

In [ ]:
son = PentatonicContinuous()
son

In [ ]:
interesting_features = ["AU01_r", "AU02_r", "AU04_r", "AU05_r", "AU06_r", "AU07_r"]
feature_display = ps.RTFeatureDisplay(interesting_features, 80)

rtdp = ps.RTDataPlayer(openface_stream, son, feature_display=feature_display)

In [ ]:
feature_display.show(fps=10)
display(son)
display(rtdp)

#### With MdaPiano

This sonification implement the same logic of the previous one, but using event-based sonification. Here, every time that the intensity of a certain AU crosses the integer thresholds (1, 2, 3, 4), a piano note with amplitude depending on the intensity is triggered. This happens both when the intensity is increasing and when it is decreasing, which can be confusing.

As OpenFace's intensity prediction can oscillate quite a lot, it is likely that some features will oscillate around grid tresholds, causing the sound to be retriggered multiple times. To alleviate this problem we will use the stream object that executes the moving average as preprocessing.

Note: MdaPiano is part of SuperCollider extensions. Make sure it's installed.

In [ ]:
class PentatonicMda(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
    fundamental = ps.MidiSliderParameter()
    
    piano_def = scn.SynthDef(
        "mdapiano",
        r"""{ |freq=440, vel=100, amp=1|
            var piano = MdaPiano.ar(
                freq,
                1,
                vel,
                decay: 0,
                release: 0,
                hard: 0,
                stereo: 0,
                mul: amp
            );
            // this can lead to artifacts, it would be better to fade out with an envelope
            DetectSilence.ar(piano, 0.05, doneAction:2);
            Out.ar(0, piano);
        }"""
    )
    
    AUs_offsets = {
        'AU01_r': 12,
        'AU02_r': 10,
        'AU04_r': 7,
        'AU05_r': 5,
        'AU06_r': 0,
        'AU07_r': 3
    }
    
    def init_parameters(self):
        self.amp = 0.3
        self.fundamental = 69
    
    @bundle
    def init_server(self):
        self.piano_def.add()

    @bundle
    def start(self):
        self.range_level = {}
        
        for label in self.AUs_offsets.keys():
            self.range_level[label] = 0
    
    @bundle
    def _process(self, row):
        for label, offset in self.AUs_offsets.items():
            intensity = row[label]
            cur_range_level = int(intensity)
            
            freq = scn.midicps(self.fundamental + offset)
            
            if cur_range_level != self.range_level[label] and cur_range_level >= 1:
                vel = scn.linlin(cur_range_level, 1, 5, 40, 127)
                scn.Synth('mdapiano', {"freq": freq, "vel": vel * self.amp})
            self.range_level[label] = cur_range_level

In [ ]:
son = PentatonicMda()
son

In [ ]:
rtdp = ps.RTDataPlayer(avg_openface_stream, son, feature_display=feature_display)

In [ ]:
feature_display.show()
rtdp

### Sonification: Drop blink

The following sonifications sonifies blinking with a synthesized drop sound. OpenFace considers the blinking active when the intensity falue is greater or equal to 1.

To reduce the effect of oscillating features, we will establish different threshold for activation and deactivation of blinking. By default, these will be 1.6 and 1, meaning that:
* blinking will switch on only when 1.6 of intensity is exceeded while the blinking was off
* blinking will switch off only when 1 of intensity is exceeded (going down) while the blinking was on

In [ ]:
class DropBlink(ps.Sonification):
    
    # hysteresis boundaries
    bounds = ps.FloatRangeSliderParameter(0, 5, 0.1)
    
    # drop definition
    drop_def = scn.SynthDef(
        "drop",
        r"""{ | freq=600, dp=1200, amp=0.5, dur=0.1, pan=0 |
            var sig, env, fch;
            fch = XLine.kr(freq, freq+dp, dur);
            sig = SinOsc.ar(fch);
            env = EnvGen.kr(Env.perc(0.001, dur, curve: -4), 1.0, doneAction: 2);
            Out.ar(0, Pan2.ar(sig, pan, env*amp))
        }"""
    )
    
    def init_parameters(self):
        self.bounds = [1, 1.6]        
    
    @bundle
    def init_server(self):
        self.drop_def.add()

    @bundle
    def start(self):
        self.blinking = False
    
    @bundle
    def stop(self):
        # drops die out alone
        pass
    
    @bundle
    def _process(self, row):
        intensity = row["AU45_r"]
        
        if self.blinking:
            if intensity < self.bounds[0]:
                self.blinking = False
        elif intensity > self.bounds[1]:
            self.blinking = True
            scn.Synth("drop")

In [ ]:
son = DropBlink()
son

In [ ]:
feature_display = ps.RTFeatureDisplay(["AU45_r"], 80)

rtdp = ps.RTDataPlayer(openface_stream, son, feature_display=feature_display)

In [ ]:
feature_display.show(10)
display(son)
rtdp

The following associates a blink to the closing of the eyes and a blink to the opening.

In [ ]:
class DoubleDropBlink(DropBlink):

    @bundle
    def _process(self, row):
        intensity = row["AU45_r"]
        
        if self.blinking:
            if intensity < self.bounds[0]:
                self.blinking = False
                scn.Synth("drop", {"freq": 900})
        elif intensity > self.bounds[1]:
            self.blinking = True
            scn.Synth("drop")

In [ ]:
son = DoubleDropBlink()
son

### Multi-percussion (event-based)

Hereafter it is shown how to implement event-based sonifications using samples.

Every sample is triggered when crossing a grid threshold; this is the same thing that we did in the PentatonicMda sonification.

Concepts:
* Sounds that usually plays together should be distinguishable
* Sounds that play more often should be less intrusive
* One area should have sounds that are somehow related

<hr>

* 1: Inner Brow Raiser - <audio controls src="samples/au01.wav"/>
* 2: Outer Brow Raiser (unilateral) - <audio controls src="samples/au02.wav"/>
* 4: Brow Lowerer - <audio controls src="samples/au04.wav"/>

<hr>

* 5: Upper Lid Raiser - <audio controls src="samples/au05.wav"/>
* 6: Cheek Raiser - <audio controls src="samples/au06.wav"/>
* 7: Lid Tightener - <audio controls src="samples/au07.wav"/>

<hr>

* 9: Nose Wrinkler (usually goes along with 4 and 10) - <audio controls src="samples/au09.wav"/>
* 10: Upper Lip Raiser - <audio controls src="samples/au10.wav"/>

<hr>

* 12: Lip Corner Puller - <audio controls src="samples/au12.wav"/>
* 14: Dimpler - <audio controls src="samples/au14.wav"/>
* 15: Lip Corner Depressor - <audio controls src="samples/au15.wav"/>
* 20: Lip Stretcher - <audio controls src="samples/au20.wav"/>

<hr>

* 23: Lip Tightener - <audio controls src="samples/au23.wav"/>

<hr>

* 17: Chin Raiser - <audio controls src="samples/au17.wav"/>
* 25: Lips Part (relax Mentalis, antagonist of AU17) - <audio controls src="samples/au25.wav"/>
* 26: Jaw Drop (usually goes along with 25) - <audio controls src="samples/au26.wav"/>

<hr>

* 28: Lip Suck (usually along with 26) - <audio controls src="samples/au28.wav"/>

    * OpenFace only provides presence information on this, which is not really reliable in a dynamic context; for this reason, we will omit it.

In [ ]:
AU01_SAMPLE_PATH = "samples/au01.wav"
AU02_SAMPLE_PATH = "samples/au02.wav"
AU04_SAMPLE_PATH = "samples/au04.wav"
AU05_SAMPLE_PATH = "samples/au05.wav"
AU06_SAMPLE_PATH = "samples/au06.wav"
AU07_SAMPLE_PATH = "samples/au07.wav"
AU09_SAMPLE_PATH = "samples/au09.wav"
AU10_SAMPLE_PATH = "samples/au10.wav"
AU12_SAMPLE_PATH = "samples/au12.wav"
AU14_SAMPLE_PATH = "samples/au14.wav"
AU15_SAMPLE_PATH = "samples/au15.wav"
AU17_SAMPLE_PATH = "samples/au17.wav"
AU20_SAMPLE_PATH = "samples/au20.wav"
AU23_SAMPLE_PATH = "samples/au23.wav"
AU25_SAMPLE_PATH = "samples/au25.wav"
AU26_SAMPLE_PATH = "samples/au26.wav"
AU28_SAMPLE_PATH = "samples/au28.wav"

#### Sonification: Percussive

In [ ]:
class Percussive(ps.Sonification):
    
    amp = ps.FloatSliderParameter(0, 1, 0.01)

    playbuf_def = scn.SynthDef(
        "playbuf",
        r"""{| out=0, bufnum=0, rate=1, amp=1, pan=0 |
            var sig;
            sig = PlayBuf.ar(1, bufnum, rate*BufRateScale.kr(bufnum), doneAction:2);
            Out.ar(0, Pan2.ar(sig, pan, amp));
        }"""
    )
    
    AUs_samples = {
        'AU01_r': AU01_SAMPLE_PATH,
        'AU02_r': AU02_SAMPLE_PATH,
        'AU04_r': AU04_SAMPLE_PATH,
        'AU05_r': AU05_SAMPLE_PATH,
        'AU06_r': AU06_SAMPLE_PATH,
        'AU07_r': AU07_SAMPLE_PATH,
        'AU09_r': AU09_SAMPLE_PATH,
        'AU10_r': AU10_SAMPLE_PATH,
        'AU12_r': AU12_SAMPLE_PATH,
        'AU14_r': AU14_SAMPLE_PATH,
        'AU15_r': AU15_SAMPLE_PATH,
        'AU17_r': AU17_SAMPLE_PATH,
        'AU20_r': AU20_SAMPLE_PATH,
        'AU23_r': AU23_SAMPLE_PATH,
        'AU25_r': AU25_SAMPLE_PATH,
        'AU26_r': AU26_SAMPLE_PATH
    }
    
    def init_parameters(self):
        self.amp = 0.3

    @bundle
    def init_server(self):
        self.playbuf_def.add()
        
        self.buffers = {}
        
        for label, sample_path in self.AUs_samples.items():
            self.buffers[label] = scn.Buffer().read(sample_path)

    @bundle
    def start(self):
        self.range_level = {}
        
        for label in self.AUs_samples.keys():
            self.range_level[label] = 0
    
    @bundle
    def stop(self):
        # synths die out alone
        pass
    
    @bundle
    def _process(self, row):
        
        for label in self.AUs_samples.keys():
            # cast intensity to integer
            cur_range_level = int(row[label])

            if cur_range_level != self.range_level[label] and cur_range_level >= 1:
                db = scn.linlin(cur_range_level, 1, 5, -40, 0, "minmax")
                scn.Synth("playbuf", {"bufnum": self.buffers[label].bufnum, "amp": scn.dbamp(db) * self.amp})

            # update old_range_level
            self.range_level[label] = cur_range_level
            
    def free(self):
        # deallocate buffers
        for buf in self.buffers.values():
            buf.free()

In [ ]:
son = Percussive()
son

In [ ]:
rtdp = ps.RTDataPlayer(openface_stream, son)
rtdp

In [ ]:
son.free()

#### Sonification: Multi-percussion sound with intensity direction

This is a refined version of the previous one. Samples will be played with different properties according to whether the intensity of the AUs is increasing (the sample will bend up) or decreasing (the sample will bend up). It is also possible to distinguish them using panning.

As we can see **init_parameters** here will take some arguments. These arguments will be passed to the function through the constructor, so that they can be specified when instantiating the sonification.

To limit the noise due to feature oscillation, we use the same approach that we used in the blinking sonification, but specifying limits that are relative to each grid level. These can be manipulated through the **bounds** parameter.

In [ ]:
class DirectionalPercussive(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
    pan = ps.CheckboxParameter()
    
    # hysteresis bounds relative to each intensity level
    bounds = ps.FloatRangeSliderParameter(-1, +1)

    playbuf_def = scn.SynthDef(
        "playbuf_bend",
        r"""{| out=0, bufnum=0, rateInitial=1, amp=1, pan=0, breakTime, rateFinal |
            var sig, rate;

            var rateAvg = (rateInitial + rateFinal) / 2;
            var sampleRateAvg = rateAvg * BufSampleRate.kr(bufnum);
            var breakFrame = breakTime * BufSampleRate.kr(bufnum);
            // mono signal: frames = samples
            var remainingFrames = BufSamples.kr(bufnum) - breakFrame;
            // calculate remaining time with dynamic rate
            var remainingTime = remainingFrames / sampleRateAvg;

            rate = EnvGen.kr(
                Env(
                    [rateInitial, rateInitial, rateFinal],
                    [breakTime, remainingTime]
                )
            );
            sig = PlayBuf.ar(1, bufnum, rate*BufRateScale.kr(bufnum), doneAction:2);
            Out.ar(0, Pan2.ar(sig, pan, amp));
        }"""
    )
    
    AuRecord = namedtuple('AuRecord', ['path', 'break_time', 'rate_up', 'rate_down'])
    
    AUs = {
        'AU01_r': AuRecord(AU01_SAMPLE_PATH, 0.2, 1.1, 0.9),
        'AU02_r': AuRecord(AU02_SAMPLE_PATH, 0.2, 1.1, 0.9),
        'AU04_r': AuRecord(AU04_SAMPLE_PATH, 0.2, 1.1, 0.9),
        'AU05_r': AuRecord(AU05_SAMPLE_PATH, 0.05, 1.1, 0.9),
        'AU06_r': AuRecord(AU06_SAMPLE_PATH, 0.03, 1.1, 0.9),
        'AU07_r': AuRecord(AU07_SAMPLE_PATH, 0.1, 1.1, 0.9),
        'AU09_r': AuRecord(AU09_SAMPLE_PATH, 0.2, 1.5, 0.7),
        'AU10_r': AuRecord(AU10_SAMPLE_PATH, 0.15, 1.2, 0.9),
        'AU12_r': AuRecord(AU12_SAMPLE_PATH, 0.1, 1.1, 0.9),
        'AU14_r': AuRecord(AU14_SAMPLE_PATH, 0.1, 1.1, 0.9),
        'AU15_r': AuRecord(AU15_SAMPLE_PATH, 0.05, 1.1, 0.9),
        'AU17_r': AuRecord(AU17_SAMPLE_PATH, 0.05, 1.5, 0.8),
        'AU20_r': AuRecord(AU20_SAMPLE_PATH, 0.1, 1.1, 0.9),
        'AU23_r': AuRecord(AU23_SAMPLE_PATH, 0.15, 1.25, 0.85),
        'AU25_r': AuRecord(AU25_SAMPLE_PATH, 0.005, 2, 0.5),
        'AU26_r': AuRecord(AU26_SAMPLE_PATH, 0.05, 2, 0.5)
    }
    
    def init_parameters(self, pan=False):
        self.amp = 0.3
        self.pan = pan
        self.bounds = [-0.3, +0.3]

    @bundle
    def init_server(self):
        self.playbuf_def.add()
        
        self.buffers = {}
        
        # allocate buffers
        for label, record in self.AUs.items():
            self.buffers[label] = scn.Buffer().read(record.path)

    @bundle
    def start(self):
        self.range_level = {}
        
        for label in self.AUs.keys():
            self.range_level[label] = 0
    
    @bundle
    def stop(self):
        # synths die out alone
        pass
    
    @bundle
    def _process(self, row):
        
        for label, record in self.AUs.items():
            intensity = row[label]
            cur_range_level = self.map_intensity(intensity, self.range_level[label])
            
            if cur_range_level != self.range_level[label] and cur_range_level >= 1:
                db = scn.linlin(cur_range_level, 1, 5, -40, 0, "minmax")
                amp = scn.dbamp(db)
            
                if cur_range_level > self.range_level[label]:
                    scn.Synth(
                        "playbuf_bend",
                        {
                            "bufnum": self.buffers[label].bufnum,
                            "amp": self.amp * amp,
                            "pan": 1 if self.pan else 0,
                            "breakTime": record.break_time,
                            "rateFinal": record.rate_up
                        }
                    )
                else:
                    scn.Synth(
                        "playbuf_bend",
                        {
                            "bufnum": self.buffers[label].bufnum,
                            "amp": self.amp * amp,
                            "pan": -1 if self.pan else 0,
                            "breakTime": record.break_time,
                            "rateFinal": record.rate_down
                        }
                    )
                    
            # update old_range_level
            self.range_level[label] = cur_range_level

    def map_intensity(self, intensity, old_level):
        cur_level = int(intensity)
        
        if cur_level == old_level:
            return cur_level        
        elif cur_level > old_level:
            return cur_level if intensity > cur_level + self.bounds[1] else old_level
        else:
            return cur_level if intensity < old_level + self.bounds[0] else old_level
            
    def free(self):
        # deallocate buffers
        for buf in self.buffers.values():
            buf.free()

In [ ]:
son = DirectionalPercussive(pan=True)
son

In [ ]:
rtdp = ps.RTDataPlayer(openface_stream, son)
rtdp

In [ ]:
son.free()

### Head

In [ ]:
feature_display = ps.RTFeatureDisplay(['pose_Rx', 'pose_Ry', 'pose_Rz'], queue_size=100)

#### Sonification: Head rotations with constant amplitude

In [ ]:
from math import pi

class Head(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
            
    def init_parameters(self):
        self.amp = 0.1
    
    @bundle
    def init_server(self):
        self.s.load_synthdefs()

    @bundle
    def start(self):
        self.synth = scn.Synth("s2", {"amp": 0, "lg": 0.015})
    
    @bundle
    def _process(self, row):
        
        pitch = scn.linlin(row['pose_Rx'], -pi/4, +pi/4, 69+12, 69-12)
        pan =   scn.linlin(row['pose_Ry'], -pi/4, +pi/4, +1, -1)
        num =   scn.linlin(row['pose_Rz'], -pi/4, +pi/4, 1, 7)
        
        self.synth.set(
            "amp", self.amp,
            "freq", scn.midicps(pitch),
            "pan", pan,
            "num", num
        )

In [ ]:
son = Head()
son

In [ ]:
rtdp = ps.RTDataPlayer(openface_stream, son, feature_display)
feature_display.show(5)
rtdp

#### Sonification: Head rotations with silent neutral position

The amplitude is mapped linearly based on the biggest rotation.
* a linear mapping makes quiet sounds more distinguishable
* every axe has a silence threshold, so that a head in a pretty neutral position does not generate any sound

In [ ]:
class SilentHead(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
    
    # max expected values for each rotation
    rx_bound = pi/4
    ry_bound = pi/4
    rz_bound = pi/4
    
    # in radians
    rx_silence_thrashold = ps.FloatSliderParameter(0, rx_bound)
    ry_silence_thrashold = ps.FloatSliderParameter(0, ry_bound)
    rz_silence_thrashold = ps.FloatSliderParameter(0, rz_bound)
    
    def init_parameters(self):
        self.amp = 0.3
        
        self.rx_silence_thrashold = self.rx_bound / 10
        self.ry_silence_thrashold = self.ry_bound / 10
        self.rz_silence_thrashold = self.rz_bound / 10
    
    @bundle
    def init_server(self):
        self.s.load_synthdefs()

    @bundle
    def start(self):
        self.synth = scn.Synth("s2", {"amp": 0, "lg": 0.015})
    
    @bundle
    def _process(self, row):
        
        rx, ry, rz = row[['pose_Rx', 'pose_Ry', 'pose_Rz']]

        amp = max(
            # clips values under silence thrashold
            scn.linlin(abs(rx), self.rx_silence_thrashold, +self.rx_bound, 0, 1, "minmax"),
            scn.linlin(abs(ry), self.ry_silence_thrashold, +self.ry_bound, 0, 1, "minmax"),
            scn.linlin(abs(rz), self.rx_silence_thrashold, +self.rz_bound, 0, 1, "minmax")
        )
        
        pitch = scn.linlin(rx, -self.rx_bound, +self.rx_bound, 69+12, 69-12)
        pan =   scn.linlin(ry, -self.ry_bound, +self.ry_bound, +1, -1)
        num =   scn.linlin(rz, -self.rz_bound, +self.rz_bound, 1, 7)
        
        self.synth.set(
            "amp", self.amp * amp,
            "freq", scn.midicps(pitch),
            "pan", pan,
            "num", num
        )

In [ ]:
son = SilentHead()
son

In [ ]:
rtdp = ps.RTDataPlayer(openface_stream, son, feature_display)
feature_display.show(5)
rtdp

#### Sonification: Noisy head rotations

In [ ]:
class NoisyHead(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
    base_tone = ps.MidiSliderParameter()

    mirror = ps.CheckboxParameter()
    
    # max expected values for each rotation
    rx_bound = ps.FloatSliderParameter(pi/2 / 10, pi/2)
    ry_bound = ps.FloatSliderParameter(pi/2 / 10, pi/2)
    rz_bound = ps.FloatSliderParameter(pi/2 / 10, pi/2)
    
    synth_def = scn.SynthDef(
        "bpf_noise",
        r"""{ | amp=1, pan=0, lg=0.5, freq=440, rq=0.2 |
            var sig;
            sig = PinkNoise.ar(amp);
            sig = BPF.ar(
                sig,
                freq.lag(lg),
                rq.lag(lg),
                // when a bandpass filter narrows, the amplitude decreases: this will balance it
                1/rq.sqrt.lag(lg)
            );
            Out.ar(0, Pan2.ar(sig, pan));
        }"""
    )
    
    def init_parameters(self):
        self.amp = 0.3
        self.base_tone = 69
        self.log_mapping = False
        self.mirror = True
        
        self.rx_bound = pi/4
        self.ry_bound = pi/4
        self.rz_bound = pi/4
    
    @bundle
    def init_server(self):
        self.synth_def.add()

    @bundle
    def start(self):
        self.synth = scn.Synth("bpf_noise", {"amp": 0, "lg": 0.015})
    
    @bundle
    def _process(self, row):
        
        rx, ry, rz = row[['pose_Rx', 'pose_Ry', 'pose_Rz']]
        
        pitch = scn.linlin(rx, -self.rx_bound, +self.rx_bound, self.base_tone+12, self.base_tone-12, "minmax")
        pan = scn.linlin(ry, -self.ry_bound, +self.ry_bound, +1, -1, "minmax")
        # linexp mapping to quality
        q = 2 ** (scn.linlin(rz, -self.rz_bound, +self.rz_bound, log2(1), log2(100), 'minmax'))
            
        if self.mirror:
            pan = -pan
            # q   = 100 - q + 1
        
        self.synth.set(
            "amp", self.amp,
            "freq", scn.midicps(pitch),
            "pan", pan,
            "rq", 1/q
        )

In [ ]:
son = NoisyHead()
son

In [ ]:
rtdp = ps.RTDataPlayer(openface_stream, son, feature_display)
# feature_display.show(5)
rtdp

### Gaze

**gaze_N_x, gaze_N_y, gaze_N_z**: Eye gaze **direction vector** in world coordinates for eye left and right eye (normalized)
- N = 0: leftmost eye in the image
- N = 1: rightmost eye in the image

**gaze_angle_x, gaze_angle_y**: Eye gaze direction in radians in world coordinates averaged for both eyes and converted into more **easy to use format** than gaze vectors.
- If a person is looking left-right this will results in the change of gaze_angle_x (from positive to negative)
- if a person is looking up-down this will result in change of gaze_angle_y (from negative to positive)
- if a person is looking straight ahead both of the angles will be close to 0 (within measurement error).

In [ ]:
feature_display = ps.RTFeatureDisplay(['gaze_angle_x', 'gaze_angle_y'], queue_size=100)

#### Sonification: Gaze with constant amplitude

In [ ]:
class Gaze(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
    
    def init_parameters(self):
        self.amp = 0.1
    
    @bundle
    def init_server(self):
        self.s.load_synthdefs()

    @bundle
    def start(self):
        self.synth = scn.Synth("s2", {"amp": 0, "lg": 0.015})
    
    @bundle
    def _process(self, row):
        
        pitch = scn.linlin(row['gaze_angle_y'], -pi/2, +pi/2, 69+12, 69-12)
        pan =   scn.linlin(row['gaze_angle_x'], -pi/2, +pi/2, +1, -1)
        
        self.synth.set(
            "amp", self.amp,
            "freq", scn.midicps(pitch),
            "pan", pan
        )

In [ ]:
son = Gaze()
son

In [ ]:
rtdp = ps.RTDataPlayer(openface_stream, son, feature_display)
feature_display.show(5)
rtdp

#### Sonification: Gaze with silent neutral position

In [ ]:
class SilentGaze(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1, 0.01)
    
    gx_silence_thrashold = ps.FloatSliderParameter(0, pi/2, 0.01)
    gy_silence_thrashold = ps.FloatSliderParameter(0, pi/2, 0.01)
    
    fundamental = ps.MidiSliderParameter()
    
    def init_parameters(self):
        self.amp = 0.1
        self.gx_silence_thrashold = pi/2 / 10
        self.gy_silence_thrashold = pi/2 / 10
        
        self.fundamental = 69
    
    @bundle
    def init_server(self):
        self.s.load_synthdefs()

    @bundle
    def start(self):
        self.synth = scn.Synth("s2", {"amp": 0, "lg": 0.015})
    
    @bundle
    def _process(self, row):
        
        gx, gy = row[['gaze_angle_x', 'gaze_angle_y']]
        
        # linear mapping
        amp = max(
            # clips values under silence thrashold
            scn.linlin(abs(gx), self.gx_silence_thrashold, pi/2, 0, 1, "minmax"),
            scn.linlin(abs(gy), self.gy_silence_thrashold, pi/2, 0, 1, "minmax")
        )
        
        pan =   scn.linlin(gx, -pi/2, +pi/2, -1, +1)
        pitch = scn.linlin(gy, -pi/2, +pi/2, self.fundamental+12, self.fundamental-12)
        
        self.synth.set(
            "amp", self.amp * amp,
            "freq", scn.midicps(pitch),
            "pan", pan
        )

In [ ]:
son = SilentGaze()
son

In [ ]:
rtdp = ps.RTDataPlayer(openface_stream, son, feature_display)
feature_display.show(5)
rtdp

### Smile

AUs regarding the smile recognised by OpenFace:
* AU06 - Cheek raiser
* AU12 - Lip Corner Puller
* AU14 - Dimpler
* AU15 - Lip Corner Depressor
* AU17 - Chin Raiser

Expected to go together:
* AU06 - AU12
* AU15 - AU17

For now we will ignore AU14 for simplicity, and we will sonify it using the percussive approach.

In [ ]:
feature_display = ps.RTFeatureDisplay(['AU06_r', 'AU12_r', 'AU15_r', 'AU17_r'], queue_size=100)

In [ ]:
class DustSmile(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1)
    base_tone = ps.MidiSliderParameter()
    
    synth_def = scn.SynthDef(
        "discrete_rev",
        r"""{ | amp=1, freq=440, density=2, mix=0.5, room=0.5, damp=0.2, lg=0.1 |
            var trig, sig, env;
            sig = SinOsc.ar(freq);
            // transform signal into short blips
            trig = Dust.kr(density);
            env = EnvGen.kr(Env.perc(0.001, 0.05), trig);
            sig = sig * env;
            sig = FreeVerb.ar(sig, mix.lag(lg), room.lag(lg), damp.lag(lg), amp.lag(lg));
            Out.ar(0, sig!2);
        }"""
    )
    
    def init_parameters(self):
        self.amp = 0.3
        self.base_tone = 69
    
    @bundle
    def init_server(self):
        self.synth_def.add()

    @bundle
    def start(self):
        self.synth = scn.Synth("discrete_rev", {"amp": 0, "density": 0})
    
    @bundle
    def _process(self, row):
        # intensities
        au06, au12, au15, au17 = row[['AU06_r', 'AU12_r', 'AU15_r', 'AU17_r']]

        if au12 > 1 and au15 > 1:
            # this should not happen usually
            return
        
        if au12 > 1:
            pitch = scn.linlin(au12, 1, 5, self.base_tone, self.base_tone+12, 'minmax')
            mix = scn.linlin(au06, 1, 3, 0.2, 1, 'minmax')
            density = scn.linlin(max(au12, au06), 1, 5, 5, 30, 'minmax')
        elif au15 > 1:
            pitch = scn.linlin(au15, 1, 5, self.base_tone, self.base_tone-12, 'minmax')
            mix = scn.linlin(au17, 1, 3, 0.2, 1, 'minmax')
            density = scn.linlin(max(au15, au17), 1, 5, 5, 30, 'minmax')
        else:
            pitch = 69
            mix = 0
            density = 0
        
        self.synth.set(
            "amp", self.amp,
            "freq", scn.midicps(pitch),
            "mix", mix,
            "density", density,
        )

In [ ]:
son = DustSmile()
son

In [ ]:
rtdp = ps.RTDataPlayer(openface_stream, son, feature_display)
feature_display.show(5)
rtdp

### Modular sonifications using GroupSonification

In [ ]:
son = ps.GroupSonification([DirectionalPercussive(), DropBlink(), NoisyHead(), SilentGaze()])
son

In [ ]:
rtdp = ps.RTDataPlayer(openface_stream, son)
rtdp

In [ ]:
son.free()

## Multi-stream

Panson provides support for processing multiple streams of data (with different frame rates) at the same time. Hereafter we show a very simple example: the amplitude of the sonification of facial features is modulated according to a stream that yields sinusoidal values at 20 fps.

In [ ]:
class NoisyHeadMulti(ps.Sonification):
    
    # parameters of the sonification
    amp = ps.FloatSliderParameter(0, 1)
    base_tone = ps.MidiSliderParameter()
    log_mapping = ps.CheckboxParameter()
    
    # max expected values for each rotation
    rx_bound = ps.FloatSliderParameter(pi/2 / 10, pi/2)
    ry_bound = ps.FloatSliderParameter(pi/2 / 10, pi/2)
    rz_bound = ps.FloatSliderParameter(pi/2 / 10, pi/2)
    
    synth_def = scn.SynthDef(
        "bpf_noise",
        r"""{ | amp=1, pan=0, lg=0.5, freq=440, rq=0.2 |
            var sig;
            sig = PinkNoise.ar(amp);
            sig = BPF.ar(
                sig,
                freq.lag(lg),
                rq.lag(lg),
                // when a bandpass filter narrows, the amplitude decreases: this will balance it
                1/rq.sqrt.lag(lg)
            );
            Out.ar(0, Pan2.ar(sig, pan));
        }"""
    )
    
    def init_parameters(self):
        self.amp = 0.3
        self.base_tone = 69
        self.log_mapping = False
        
        self.rx_bound = pi/2
        self.ry_bound = pi/2
        self.rz_bound = pi/4
    
    @bundle
    def init_server(self):
        self.synth_def.add()

    @bundle
    def start(self):
        self.synth = scn.Synth("bpf_noise", {"amp": 0, "lg": 0.015})
    
    @bundle
    def _process(self, row):
        
        # use sin value as amplitude modulator
        amp_mod = scn.linlin(row['value'], -1, 1, 0, 1, 'minmax')
        
        rx, ry, rz = row[['pose_Rx', 'pose_Ry', 'pose_Rz']]
        
        # use log() - 1 to map
        if self.log_mapping:
            rx_log = log2(scn.linlin(abs(rx), 0, self.rx_bound, 1, 2, 'minmax'))
            rx_midi = scn.linlin(rx_log, 0, 1, 0, 12, 'minmax')
            sign = 1 if rx >= 0 else -1
            pitch = self.base_tone + sign * rx_midi
            
            ry_exp = scn.linlin(abs(ry), 0, self.ry_bound, 1, 2, 'minmax')
            sign = 1 if ry >= 0 else -1
            pan = sign * log2(ry_exp)
                        
            q =   scn.linlin(rz, -self.rz_bound, +self.rz_bound, 1, 100, 'minmax')
            
        else:
            pitch = scn.linlin(rx, -self.rx_bound, +self.rx_bound, self.base_tone+12, self.base_tone-12, "minmax")
            pan =   scn.linlin(ry, -self.ry_bound, +self.ry_bound, +1, -1, "minmax")
            # linexp mapping to quality
            q =   exp(scn.linlin(rz, -self.rz_bound, +self.rz_bound, log2(1), log2(100), 'minmax'))
        
        self.synth.set(
            "amp", self.amp * amp_mod,
            "freq", scn.midicps(pitch),
            "pan", pan,
            "rq", 1/q
        )

In [ ]:
son = NoisyHeadMulti()
son

In [ ]:
sin = streams.DummySin('sin', kwargs={'fps': 500, 'timestamps': False}).test()

Using **multi-threading** approach

In [ ]:
rtdpmt = ps.RTDataPlayerMT([openface_stream, sin], son, fps=30)

In [ ]:
rtdpmt

Using **multi-processing** approach.

In [ ]:
rtdpmp = ps.RTDataPlayerMP([openface_stream, sin], son, fps=30)

In [ ]:
rtdpmp